In [1]:
import tensorflow as tf
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
(train_image,train_label),(test_image,test_label) = tf.keras.datasets.fashion_mnist.load_data()

In [3]:
train_image.shape

(60000, 28, 28)

In [4]:
train_label.shape

(60000,)

In [5]:
# 对数据进行归一化处理
train_image = train_image/255
test_image = test_image/255

In [6]:
# 建立模型
model = tf.keras.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=(28,28)))
model.add(tf.keras.layers.Dense(128,activation='relu'))
model.add(tf.keras.layers.Dense(10,activation='softmax'))

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               100480    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


In [8]:
# 模型编译
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['acc'])

In [9]:
# 训练模型
model.fit(train_image,train_label,epochs=3)

Epoch 1/3
1875/1875 [==============================] - 4s 2ms/step - loss: 0.5024 - acc: 0.8239
Epoch 2/3
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3745 - acc: 0.8642
Epoch 3/3
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3361 - acc: 0.8760


In [10]:
# 模型评价 verbose=1显示评价过程，verbose=0不显示
model.evaluate(test_image,test_label,verbose=0)

[0.37809574604034424, 0.8626000285148621]

# 保存整个模型

In [11]:
# 保存模型，参数为保存路径
model.save('less_model.h5')

In [12]:
# 使用保存好的模型
new_model = tf.keras.models.load_model('less_model.h5')

In [13]:
new_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               100480    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


In [14]:
new_model.evaluate(test_image,test_label,verbose=0)

[0.37809574604034424, 0.10790000110864639]

# 仅保存架构

In [15]:
# 保存模型架构
json_config = model.to_json()

In [16]:
json_config

'{"class_name": "Sequential", "config": {"name": "sequential", "layers": [{"class_name": "InputLayer", "config": {"batch_input_shape": [null, 28, 28], "dtype": "float32", "sparse": false, "ragged": false, "name": "flatten_input"}}, {"class_name": "Flatten", "config": {"name": "flatten", "trainable": true, "batch_input_shape": [null, 28, 28], "dtype": "float32", "data_format": "channels_last"}}, {"class_name": "Dense", "config": {"name": "dense", "trainable": true, "dtype": "float32", "units": 128, "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}}, {"class_name": "Dense", "config": {"name": "dense_1", "trainable": true, "dtype": "float32", "units": 10, "activation": "softmax", "use_bias": true, "kernel_initializer": {"class_

In [17]:
# 重建模型
reinitialized_model = tf.keras.models.model_from_json(json_config)

In [18]:
reinitialized_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               100480    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


In [21]:
reinitialized_model.evaluate(test_image,test_label,verbose=0)

[2.4520509243011475, 0.1062999963760376]

In [20]:
reinitialized_model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['acc'])

# 仅保存模型权重

In [22]:
# 获取模型权重
weights = model.get_weights()

In [23]:
# 为模型设置权重
reinitialized_model.set_weights(weights)

In [24]:
reinitialized_model.evaluate(test_image,test_label,verbose=0)

[0.37809574604034424, 0.8626000285148621]

In [25]:
# 将模型权重保存到磁盘
model.save_weights('less_weights.h5')

In [26]:
# 从磁盘中加载保存好的权重
reinitialized_model.load_weights('less_weights.h5')

In [27]:
reinitialized_model.evaluate(test_image,test_label,verbose=0)

[0.37809574604034424, 0.8626000285148621]

# 在训练期间保存检查点

In [33]:
# 配置保存检查点的回调函数
cp_callback = tf.keras.callbacks.ModelCheckpoint('training_cp/cp.ckpt',save_weights_only=True)

In [34]:
# 建立模型
model = tf.keras.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=(28,28)))
model.add(tf.keras.layers.Dense(128,activation='relu'))
model.add(tf.keras.layers.Dense(10,activation='softmax'))

In [35]:
# 模型编译
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['acc'])

In [36]:
# 使用保存好的检查点
model.load_weights('training_cp/cp.ckpt')

In [32]:
# 训练模型，指定保存检查点的回调函数
model.fit(train_image,train_label,epochs=3,callbacks=[cp_callback])

Epoch 1/3
1875/1875 [==============================] - 5s 3ms/step - loss: 0.5029 - acc: 0.8213
Epoch 2/3
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3758 - acc: 0.8645
Epoch 3/3
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3358 - acc: 0.8771


# 自定义训练中保存检查点

In [37]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=(28,28)))
model.add(tf.keras.layers.Dense(128,activation='relu'))
model.add(tf.keras.layers.Dense(10))

In [38]:
optimizer = tf.keras.optimizers.Adam()

In [39]:
loss_func = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [40]:
def loss(model,x,y):
    y_ = model(x)
    return loss_func(y,y_)

In [44]:
train_loss = tf.keras.metrics.Mean('train_loss',dtype=tf.float32)
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('train_accuracy')
test_loss = tf.keras.metrics.Mean('test_loss',dtype=tf.float32)
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('test_accuracy')

In [45]:
def train_step(model,images,labels):
    with tf.GradientTape() as t:
        pred = model(images)
        loss_step = loss_func(labels,pred)
    grads = t.gradient(loss_step,model.trainable_variables)
    optimizer.apply_gradients(zip(grads,model.trainable_variables))
    train_loss(loss_step)
    train_accuracy(labels,pred)

In [46]:
dataset = tf.data.Dataset.from_tensor_slices((train_image,train_label))

In [47]:
dataset = dataset.shuffle(10000).batch(32)

In [48]:
# 指定检查点保存位置
cp_dir = './customtrain_cp'
cp_prefix = os.path.join(cp_dir,'ckpt')

In [49]:
# 定义检查点，保存优化器和模型架构
checkpoint = tf.train.Checkpoint(optimizer=optimizer,model=model)

In [52]:
def train():
    for epoch in range(5):
        for (batch,(images,labels)) in enumerate(dataset):
            train_step(model,images,labels)
        print('Epoch{} loss is {}'.format(epoch,train_loss.result()))
        print('Epoch{} Accuracy is {}'.format(epoch,train_accuracy.result()))
        train_loss.reset_states()
        train_accuracy.reset_states()
        if(epoch + 1)%2 == 0:
            # 每两个epoch保存检查点
            checkpoint.save(file_prefix = cp_prefix)

In [53]:
train()

Epoch0 loss is 0.3365711569786072
Epoch0 Accuracy is 0.8775333166122437
Epoch1 loss is 0.31290820240974426
Epoch1 Accuracy is 0.8865500092506409
Epoch2 loss is 0.29701972007751465
Epoch2 Accuracy is 0.8906333446502686
Epoch3 loss is 0.2798936069011688
Epoch3 Accuracy is 0.8957666754722595
Epoch4 loss is 0.27014732360839844
Epoch4 Accuracy is 0.8999166488647461


In [54]:
# 取出检查点目录下最新的检查点
tf.train.latest_checkpoint(cp_dir)

'./customtrain_cp\\ckpt-2'

In [55]:
# 从指定检查点恢复检查点
checkpoint.restore(tf.train.latest_checkpoint(cp_dir))